In [1]:
import pandas as pd
import numpy as np
from joblib import load
from pathlib import Path

DATA_PROCESSED = Path("../data/processed")
OUT_TABLES = Path("../outputs/tables")
OUT_TABLES.mkdir(parents=True, exist_ok=True)


In [3]:
df = pd.read_csv(DATA_PROCESSED / "model_data.csv")

model = load("../outputs/models/demand_prediction_model.pkl")



In [ ]:
latest_week = df["week"].max()
baseline = df[df["week"] == latest_week].copy()

In [6]:
# helper function
def compute_impact(df_new, df_base):
    result = {}
    result["units"] = df_new["units_sold_pred"].sum()
    result["revenue"] = (df_new["units_sold_pred"] * df_new["final_price"]).sum()
    result["profit"] = (
        df_new["units_sold_pred"] * (df_new["final_price"] - df_new["cost_price"])
    ).sum()
    
    result["units_change_pct"] = (
        (result["units"] - df_base["units"].sum()) / df_base["units"].sum()
    ) * 100
    
    result["revenue_change_pct"] = (
        (result["revenue"] - df_base["revenue"].sum()) / df_base["revenue"].sum()
    ) * 100
    
    result["profit_change_pct"] = (
        (result["profit"] - df_base["profit"].sum()) / df_base["profit"].sum()
    ) * 100
    
    return result


In [7]:
features = ["final_price", "discount_percent", "marketing_spend", "week"]

baseline["units_sold_pred"] = model.predict(baseline[features])

baseline_summary = {
    "units": baseline["units_sold_pred"].sum(),
    "revenue": (baseline["units_sold_pred"] * baseline["final_price"]).sum(),
    "profit": (
        baseline["units_sold_pred"] * (baseline["final_price"] - baseline["cost_price"])
    ).sum()
}

baseline_summary


{'units': np.float64(12484.703832948146),
 'revenue': np.float64(419898.33111371717),
 'profit': np.float64(106777.10455356224)}

# Price fluctuation
Say if we increase Price by 5%, we will then predict the number of units sold.
The hypothesis is gonna be that 
- The number of units sold would decrease.


We see that the number of units sold has decreased but the revenue has increased which is a good thing so even the number of customer's has decreased - we can say that since revenue has increased - its a win win

And hence profit also has increased

Hypothesis has been tested but units sold decreasing is not a bad thing




In [8]:
scenario_price_up = baseline.copy()
scenario_price_up["final_price"] *= 1.05


In [9]:
scenario_price_up["units_sold_pred"] = model.predict(
    scenario_price_up[features]
)

impact_price_up = compute_impact(
    scenario_price_up, pd.DataFrame([baseline_summary])
)

impact_price_up


{'units': np.float64(12714.809136842206),
 'revenue': np.float64(447106.49047822296),
 'profit': np.float64(130064.62393017687),
 'units_change_pct': np.float64(1.843097817721509),
 'revenue_change_pct': np.float64(6.4797017154939),
 'profit_change_pct': np.float64(21.809468868799478)}

# Discount Fluctuation

### Reduce discount by 3%

Hypothesis - THe number of units sold, revenue and profit

In [10]:
scenario_discount_down = baseline.copy()
scenario_discount_down["discount_percent"] = np.maximum(
    scenario_discount_down["discount_percent"] - 3, 0
)


In [11]:
scenario_discount_down["units_sold_pred"] = model.predict(
    scenario_discount_down[features]
)

impact_discount_down = compute_impact(
    scenario_discount_down, pd.DataFrame([baseline_summary])
)

impact_discount_down


{'units': np.float64(12180.934244221107),
 'revenue': np.float64(410261.8479053545),
 'profit': np.float64(104962.33766068045),
 'units_change_pct': np.float64(-2.4331341198929026),
 'revenue_change_pct': np.float64(-2.294956301160648),
 'profit_change_pct': np.float64(-1.6995842886631676)}

# Marketing fluctuation
### Increase marketing spend by 10%

In [12]:
scenario_marketing_up = baseline.copy()
scenario_marketing_up["marketing_spend"] *= 1.10

scenario_marketing_up["units_sold_pred"] = model.predict(
    scenario_marketing_up[features]
)

impact_marketing_up = compute_impact(
    scenario_marketing_up, pd.DataFrame([baseline_summary])
)

impact_marketing_up


{'units': np.float64(12611.542259332218),
 'revenue': np.float64(424265.9436250814),
 'profit': np.float64(108000.05032370138),
 'units_change_pct': np.float64(1.0159506231083808),
 'revenue_change_pct': np.float64(1.040159530946404),
 'profit_change_pct': np.float64(1.1453258404526923)}

In [13]:
scenario_results = pd.DataFrame([
    {"scenario": "Price +5%", **impact_price_up},
    {"scenario": "Discount -3%", **impact_discount_down},
    {"scenario": "Marketing +10%", **impact_marketing_up},
])

scenario_results


,scenario,units,revenue,profit,units_change_pct,revenue_change_pct,profit_change_pct
0,Price +5%,12714.809137,447106.490478,130064.623930,1.843098,6.479702,21.809469
1,Discount -3%,12180.934244,410261.847905,104962.337661,-2.433134,-2.294956,-1.699584
2,Marketing +10%,12611.542259,424265.943625,108000.050324,1.015951,1.040160,1.145326


In [14]:
scenario_results.to_csv(
    OUT_TABLES / "scenario_results.csv",
    index=False
)

print("Saved scenario_results.csv")


Saved scenario_results.csv
